# Bike Sales Project

In this project I decided to explore the dataset that consist of sales data of several countries of different bikes, their accessoires and some clothes.

The data source:
https://www.kaggle.com/sadiqshah/bike-sales-in-europe

Such exploration can be useful for marketing purposes, that is why it was interesting for me to try to make some research of the dataset.

In [1]:
import pandas as pd
import altair as alt
import geopandas as gpd

In [2]:
df = pd.read_csv('Sales.csv')

### Let's make some overview of the dataset.

In [3]:
df.head()

,Date,Day,Month,Year,Customer_Age,Age_Group,Customer_Gender,Country,State,Product_Category,Sub_Category,Product,Order_Quantity,Unit_Cost,Unit_Price,Profit,Cost,Revenue
0,2013-11-26,26,November,2013,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
1,2015-11-26,26,November,2015,19,Youth (<25),M,Canada,British Columbia,Accessories,Bike Racks,Hitch Rack - 4-Bike,8,45,120,590,360,950
2,2014-03-23,23,March,2014,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,23,45,120,1366,1035,2401
3,2016-03-23,23,March,2016,49,Adults (35-64),M,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,20,45,120,1188,900,2088
4,2014-05-15,15,May,2014,47,Adults (35-64),F,Australia,New South Wales,Accessories,Bike Racks,Hitch Rack - 4-Bike,4,45,120,238,180,418


### We have data from 6 countries and 6 years(2011-2016):

In [4]:
df.Country.unique()

array(['Canada', 'Australia', 'United States', 'Germany', 'France',
       'United Kingdom'], dtype=object)

In [5]:
df.Year.unique()

array([2013, 2015, 2014, 2016, 2012, 2011])

### Also we have 3 Product categories:

In [6]:
df.Product_Category.unique()

array(['Accessories', 'Clothing', 'Bikes'], dtype=object)

In [7]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### What is the genaral Revenue according to country during different years?

At first, I wanted to explore what country had the largest revenue from our products and how their revenue changed over years.

I decided to use bar chart with the ability to change the year to visualize.
It is very convinient to compare revenue among different countries with this type of chart, also color scales help to see the difference in values.
Besides, we can clearly see how revenues changed over years as we have the same scale for all the years.

Also, I thought to use the line chart, where color would represent each country,
but with barchart it is easy to see what values belongs to each country, with line chart it would be more difficult.

In [8]:
df1 = df[['Year', 'Country', 'Revenue']]

In [9]:
df1 = df1.groupby(['Year', 'Country']).sum()['Revenue'].reset_index() 

In [10]:
input_slider = alt.binding_range(min=df1.Year.min(), max=2016, step=1, name='Select year:')
select_year = alt.selection_single(name="year", fields = ['Year'], bind=input_slider, init = {'Year': 2011})

In [11]:
c1 = alt.Chart(df1).mark_bar().encode(
    x = alt.X('Country:N',
             axis = alt.Axis(titleFontSize = 18, 
                             labelAngle = 0, 
                             labelFontSize = 13, 
                             titleY = 30)),
    y = alt.Y('Revenue:Q', 
             axis = alt.Axis(format = '.1s', 
                             titleFontSize = 18, 
                             titleX = -50),
             scale = alt.Scale(domain = [df1.Revenue.min(), df1.Revenue.max()])),
    color=alt.Color('Revenue:Q',
                    scale = alt.Scale(scheme = 'goldgreen', 
                                      domain = [df1.Revenue.min(), df1.Revenue.max()]),
                    legend = alt.Legend(format = '.1s',
                                        titleFontSize = 15)),
    tooltip = alt.Tooltip('Revenue:Q', 
                          format = '.2s')
).transform_filter(select_year
).add_selection(select_year
).properties(
    title = 'Product revenue in different countries by years',
    width = 800, 
    height = 600,
    background = '#F9F9F9',
    padding = 25
).configure_title(fontSize = 20,
                  dy = -25)

In [34]:
c1

alt.Chart(...)

With this chart we see that the USA always had the largest revenue among all, also 2015 year was the most successfull regarding sales for all the countries.

## What was the Revenue structure of different Categories?

The next step was to discover what category brings us the most of money. And how the revenues from each category changed over years.

Here I decided to use point chart, where size and color represent the revenue.
With this diagram we clearly see the most popular category by the revenues, but it is harder to understand the difference in values.

In this question, I also thought about the bar chart or arear plot.
These charts also would be suitable for this question. Maybe it would be even easier to compare the revenue between the categories with bar chart.

In [13]:
df2 = df[['Year', 'Product_Category', 'Revenue']]

In [14]:
df2 = df2.groupby(['Year', 'Product_Category']).sum()['Revenue'].reset_index() 

In [15]:
c2 = alt.Chart(df2).mark_point(filled = True).encode(
    x = alt.X('Year:N',
             axis = alt.Axis(titleFontSize = 18,
                             labelAngle = 0,
                             labelFontSize = 14,
                             titleY = 40)),
    y = alt.Y('Product_Category:N',
             axis = alt.Axis(title = 'Product Category',
                             titleFontSize = 18,
                             labelFontSize = 14)),
    color=alt.Color('Revenue:Q', 
                    scale = alt.Scale(scheme = 'viridis'), 
                    legend = alt.Legend(format = '.1s')),
    size = alt.Size('Revenue:Q',
                    scale = alt.Scale(range = [1000, 5000])),
    tooltip = alt.Tooltip('Revenue:Q',
                          format = '.3s')
).properties(title = 'Product revenue from different categories by years',
            width = 800,
            height = 600,
            background = '#F9F9F9',
            padding = 25
).configure_title(fontSize = 22, 
                  dy = -25)

In [35]:
c2

alt.Chart(...)

With this diagram we see that Bikes category always was the most popular, besides the products from the other two categories started to sell only from 2013

## How Revenue structure vary in different countries?

I also wanted to discover how revenues from the categories varied in different countries.

To answer this question, I used the bar chart for each country. Each color represent the category.

Here we could also use stacked bar chart, but with this type of diagram, it wouldn't be convinient to compare revenues from the same category between different countries.

In [17]:
df3 = df[['Country', 'Product_Category', 'Revenue']]

In [18]:
df3 = df3.groupby(['Country', 'Product_Category']).sum()['Revenue'].reset_index()

In [19]:
c3 = alt.Chart(df3).mark_bar().encode(
    x = alt.X('Product_Category:N',
             axis = alt.Axis(title= '',
                             labelAngle = 0,
                             labelFontSize = 11)),
    y = alt.Y('Revenue:Q',
             axis = alt.Axis(format = '.1s',
                             titleFontSize = 18,
                             titleX = -50)),
    column = alt.Column('Country:N',
                        title = ''),
    color=alt.Color('Product_Category:N',
                   legend = alt.Legend(title = 'Product Category',
                                       titleFontSize = 17,
                                       labelFontSize = 16)),
        tooltip = alt.Tooltip('Revenue:N',
                             format = '.3s'), 
).properties(title = 'Product revenue in different countries by categories',
             width = 950/len(pd.unique(df3.Country)), 
             height = 600,
             background = '#F9F9F9',
             padding = 30
).configure_title(fontSize = 20, anchor= 'middle', dy = -25)

In [36]:
c3

alt.Chart(...)

## What is the most popular Age Group in each country?

In marketing, researches about the popularity of each age group are very widely used. So, it was interesting for me to discover this question.
Also I decided to divide data by gender characteristic.

Here I used the heatmap. With this chart it is easy to see the most popular age group in each country as it is the darkest shade of the color.
The clear difference between values is harder to observe with the heatmap.
However, this diagram was understandable for me when answering this question. 

In [21]:
df4 = df[['Country', 'Age_Group', 'Customer_Gender', 'Revenue']]

In [22]:
df4 = df4.groupby(['Country', 'Age_Group', 'Customer_Gender']).mean()['Revenue'].reset_index() 

In [23]:
df4_f = df4[df4.Customer_Gender == 'F']
df4_m = df4[df4.Customer_Gender == 'M']

In [24]:
c4_1 = alt.Chart(df4_f).mark_rect().encode(
    x = alt.X('Country:O',
             axis = alt.Axis(labelAngle = 0,
                             labelFontSize = 13,
                             titleFontSize = 16,
                             titleY = 35)),
    y = alt.Y('Age_Group:O',
             axis = alt.Axis(title = 'Age Group',
                             labelFontSize = 13, 
                             titleFontSize = 16, 
                             titleX = -150)),
    color = alt.Color('Revenue:Q',
                      legend = alt.Legend(title = 'Revenue',
                                          titleFontSize = 17), 
                      scale = alt.Scale(scheme = 'yellowgreen')),
    tooltip = alt.Tooltip('Revenue:Q',
                          format = '.3s')
).properties(title = 'Product ravenue according to the age group in different countries (FEMALE)')

In [25]:
c4_2 = alt.Chart(df4_m).mark_rect().encode(
    x = alt.X('Country:O',
             axis = alt.Axis(labelAngle = 0,
                             labelFontSize = 13,
                             titleFontSize = 16,
                             titleY = 35)),
    y = alt.Y('Age_Group:O',
             axis = alt.Axis(title = 'Age Group',
                             labelFontSize = 13,
                             titleFontSize = 16,
                             titleX = -150)),
    color = alt.Color('Revenue:Q',
                      legend = alt.Legend(title = 'Revenue',
                                          titleFontSize = 17),
                      scale = alt.Scale(scheme = 'yellowgreen')),
    tooltip = alt.Tooltip('Revenue:Q',
                          format = '.3s')
).properties(title = 'Product ravenue according to the age group in different countries (MALE)')

In [26]:
c4 = alt.hconcat(c4_1.properties(width = 600, height = 400), 
            c4_2.properties(width = 600, height = 400)
).properties(background = '#F9F9F9',
             padding = 25
).configure_title(fontSize = 18, 
                  dy = -25
).configure_concat(spacing = 35)

In [37]:
c4

alt.HConcatChart(...)

It was very interesting to discover that Seniors group brings the least revenues in all countries except Australia.

## What is the revenue structure in different age group by category?

Here I decided to use the bar chart with the ability to change the product category so that we could see the most popular age group between the categories.
Also we can see how each age group differs in revenue over the categories.
This type of diagram was the most convinient to answer the question and to make research about the revenue structure in age groups.

In [28]:
df5 = df[['Age_Group', 'Product_Category','Revenue']]

In [29]:
df5 = df5.groupby(['Product_Category','Age_Group']).sum()['Revenue'].reset_index() 

In [30]:
input_dropdown = alt.binding_select(options = df5.Product_Category.unique())
select_category = alt.selection_single(fields = ['Product_Category'], bind = input_dropdown, name = 'Category:', init = {'Product_Category': 'Bikes'})

In [31]:
c5 = alt.Chart(df5).mark_bar().encode(
    x = alt.X('Age_Group:N',
             axis = alt.Axis(title = 'Age Group',
                             titleFontSize = 18, 
                             labelAngle = 0,
                             labelFontSize = 14,
                             titleY = 40)),
    y = alt.Y('Revenue:Q',
             axis = alt.Axis(titleFontSize = 18,
                             labelFontSize = 14,
                             titleX = -70,
                             format = '.1s'),
             scale = alt.Scale(domain = [df5.Revenue.min(), df5.Revenue.max()])),
    color=alt.Color('Revenue:Q',
                   legend = alt.Legend(title = 'Revenue', 
                                       titleFontSize = 17,
                                       format = '.1s')),
                   tooltip = alt.Tooltip('Revenue:Q', 
                                        format = '.3s')
).transform_filter(select_category
).add_selection(select_category
).properties(title = 'Product ravenue by age group',
             width = 800,
             height = 600,
             background = '#F9F9F9',
             padding = 25
).configure_title(fontSize = 18,
                  dy = -25)

In [38]:
c5

alt.Chart(...)

We can conclude that adult people of 35-64 years old are the most interested in buying our products.